# scribbles
here is a scratchpad for code that isn't quite ready to stand alone in the directories or in it's own script...

In [1]:
# Used to read/manipulate netCDF data
import xarray as xr

# Used to convert to .idx
from OpenVisus import *

# Used for numerical work
import numpy as np

# Used for processing netCDF time data
import datetime

# Used for interacting with OS file system (to get directory file names)
import os

# # Used for redownloading netCDF files from https://firesmoke.ca/ if opening currently downloaded file fails
# import wget

# Accessory, used to generate progress bar for running for loops
from tqdm.notebook import tqdm

In [2]:
# firesmoke_dir = "/Users/arleth/Mount/firesmoke"
firesmoke_dir = "/usr/sci/cedmav/data/firesmoke"

ids = ["BSC18CA12-01", "BSC00CA12-01", "BSC06CA12-01", "BSC12CA12-01"]

In [3]:
# for parsing time flags (TFLAG) from netcdf files
def parse_tflag(tflag):
    year = int(tflag[0] // 1000)
    day_of_year = int(tflag[0] % 1000)
    date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=day_of_year - 1)

    time_in_day = int(tflag[1])
    hours = time_in_day // 10000
    minutes = (time_in_day % 10000) // 100
    seconds = time_in_day % 100

    full_datetime = datetime.datetime(year, date.month, date.day, hours, minutes, seconds)
    return full_datetime

In [4]:
file18 = f'{firesmoke_dir}/{ids[0]}/dispersion_20210304.nc'
file00 = f'{firesmoke_dir}/{ids[1]}/dispersion_20210304.nc'
file06 = f'{firesmoke_dir}/{ids[2]}/dispersion_20210304.nc'
file12 = f'{firesmoke_dir}/{ids[3]}/dispersion_20210304.nc'

file_mine = f'{firesmoke_dir}/{ids[1]}/dispersion_20240209.nc'

In [5]:
ds00 = xr.open_dataset(file00)
ds06 = xr.open_dataset(file06)
ds12 = xr.open_dataset(file12)
ds18 = xr.open_dataset(file18)

ds_mine = xr.open_dataset(file_mine)

In [6]:
time18 = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds18['TFLAG'].values)]
time18[0:6]

['2021-03-04 03:00:00',
 '2021-03-04 04:00:00',
 '2021-03-04 05:00:00',
 '2021-03-04 06:00:00',
 '2021-03-04 07:00:00',
 '2021-03-04 08:00:00']

In [7]:
time00 = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds00['TFLAG'].values)]
time00[0:6]

['2021-03-04 09:00:00',
 '2021-03-04 10:00:00',
 '2021-03-04 11:00:00',
 '2021-03-04 12:00:00',
 '2021-03-04 13:00:00',
 '2021-03-04 14:00:00']

In [8]:
time06 = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds06['TFLAG'].values)]
time06[0:6]

['2021-03-04 15:00:00',
 '2021-03-04 16:00:00',
 '2021-03-04 17:00:00',
 '2021-03-04 18:00:00',
 '2021-03-04 19:00:00',
 '2021-03-04 20:00:00']

In [9]:
time12 = [parse_tflag(t).strftime("%Y-%m-%d %H:%M:%S") for t in np.squeeze(ds12['TFLAG'].values)]
time12[0:6]

['2021-03-04 21:00:00',
 '2021-03-04 22:00:00',
 '2021-03-04 23:00:00',
 '2021-03-05 00:00:00',
 '2021-03-05 01:00:00',
 '2021-03-05 02:00:00']

In [11]:
print(f'time00: {time00[0]} to {time00[-1]}')
print(f'time06: {time06[0]} to {time06[-1]}')
print(f'time12: {time12[0]} to {time12[-1]}')
print(f'time18: {time18[0]} to {time18[-1]}')

time00: 2021-03-04 09:00:00 to 2021-03-06 11:00:00
time06: 2021-03-04 15:00:00 to 2021-03-06 17:00:00
time12: 2021-03-04 21:00:00 to 2021-03-06 23:00:00
time18: 2021-03-04 03:00:00 to 2021-03-05 23:00:00


In [1]:
print(f"time00: {datetime.datetime.strptime(time00[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(time00[-1], '%Y-%m-%d %H:%M:%S')}")
print(f"time06: {datetime.datetime.strptime(time06[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(time06[-1], '%Y-%m-%d %H:%M:%S')}")
print(f"time12: {datetime.datetime.strptime(time12[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(time12[-1], '%Y-%m-%d %H:%M:%S')}")
print(f"time18: {datetime.datetime.strptime(time18[0], '%Y-%m-%d %H:%M:%S') - datetime.datetime.strptime(time18[-1], '%Y-%m-%d %H:%M:%S')}")

NameError: name 'datetime' is not defined

In [12]:
print(f"file00: {file00}")
print(f"file06: {file06}")
print(f"file12: {file12}")
print(f"file18: {file18}")

file00: /usr/sci/cedmav/data/firesmoke/BSC00CA12-01/dispersion_20210304.nc
file06: /usr/sci/cedmav/data/firesmoke/BSC06CA12-01/dispersion_20210304.nc
file12: /usr/sci/cedmav/data/firesmoke/BSC12CA12-01/dispersion_20210304.nc
file18: /usr/sci/cedmav/data/firesmoke/BSC18CA12-01/dispersion_20210304.nc
